# Blue Brain BioExplorer
![](../../bioexplorer_banner.png)

In [ ]:
from bioexplorer import BioExplorer
from mediamaker import MovieMaker
import os

hostname = 'localhost:5000'
resource_folder = '../../../tests/test_files/'
    
be = BioExplorer(hostname)
core = be.core_api()
mm = MovieMaker(be)
version = be.version()
print('Version ' + version)

sequence_name = 'low_glucose'

## Data loading

In [ ]:
'''Reset current scene'''
status = be.reset()

In [ ]:
'''Accelerate loading by not showing models as they are loaded'''
status = be.set_general_settings(
    logging_enabled=True,
    model_visibility_on_creation=False)

In [ ]:
cache_folder = resource_folder + 'caches/' + version + '/'

In [ ]:
cache_filename = cache_folder + sequence_name + '_scenario.bioexplorer'
print('Loading ' + cache_filename)
dummy = core.add_model(cache_filename)

In [ ]:
nb_models = len(core.scene.models)
if nb_models != 152:
    print('ERROR: Not all models have been loaded!!! Expected 152, found ' + str(nb_models))
else:
    print('All good!')

In [ ]:
'''Show loaded models'''
status = be.set_models_visibility(True)

## Rendering settings

In [ ]:
status = core.set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer', head_light=False,
    samples_per_pixel=1, subsampling=1, max_accum_frames=128)
params = be.core_api().BioExplorerRendererParams()
params.exposure = 1.0
params.gi_samples = 1
params.gi_weight = 0.3
params.gi_distance = 5000
params.shadows = 1.0
params.soft_shadows = 0.02
params.fog_start = 1000
params.fog_thickness = 300
params.max_bounces = 1
params.use_hardware_randomizer = True
status = core.set_renderer_params(params)

status = core.clear_lights()
status = core.add_light_directional(
    angularDiameter=0.5, color=[1,1,1], direction=[-0.7,-0.4,-1],
    intensity=1.0, is_visible=False
)

## Image settings

In [ ]:
image_size = [3840, 2160]
image_samples_per_pixel = 64

output_folder = resource_folder + 'frames/' + \
                version + '/' + sequence_name + '/'
print(output_folder)

command_line='mkdir -p ' + output_folder
os.system(command_line)
command_line='ls ' + output_folder

if os.system(command_line) != 0:
    print('ERROR: Failed to create output folder')

## Snapshots

In [ ]:
def set_all_materials_to_grey():
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)
        palette = list()
        for p in range(nb_materials):
            palette.append([0.25,0.25,0.25])
        be.set_materials(
            model_ids=[model_id], material_ids=material_ids, 
            diffuse_colors=palette, specular_colors=palette)

In [ ]:
def set_glycans_materials(high_mannose=False, complex=False, hybrid=False, o_glycans=False):
    glycans_colors = [[0, 1, 1], [1, 1, 0], [1, 0, 1], [0.2, 0.2, 0.7]]

    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)
        if be.NAME_GLYCAN_HIGH_MANNOSE in model_name and high_mannose:
            palette = list()
            for m in range(nb_materials):
                palette.append(glycans_colors[0])
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette
            )
        if be.NAME_GLYCAN_COMPLEX in model_name and complex:
            palette = list()
            for m in range(nb_materials):
                palette.append(glycans_colors[1])
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette
            )            
        if be.NAME_GLYCAN_HYBRID in model_name and hybrid:
            palette = list()
            for m in range(nb_materials):
                palette.append(glycans_colors[2])
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette
            )                
        if be.NAME_GLYCAN_O_GLYCAN in model_name and o_glycans:
            palette = list()
            for m in range(nb_materials):
                palette.append(glycans_colors[3])
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette
            )

### Frame 600 - Spike and glycans

In [ ]:
status = core.set_camera(
    orientation=[-0.005, 0.047, -0.004, 0.998],
    position=[211.585, 74.499, -106.416],
    target=[201.490, 74.488, -107.411]
)

In [ ]:
set_all_materials_to_grey()
mm.create_snapshot(
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/00600.png')

In [ ]:
set_glycans_materials(complex=True)
mm.create_snapshot(
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/00600a.png')

In [ ]:
set_glycans_materials(o_glycans=True)
mm.create_snapshot(
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/00600b.png')

In [ ]:
set_glycans_materials(high_mannose=True)
mm.create_snapshot(
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/00600c.png')

### Frame 1600 - Immune system

In [ ]:
status = core.set_camera(
    orientation=[-0.003, 0.243, -0.003, 0.970],
    position=[238.163, 46.437, 372.585],
    target=[237.693, 46.431, 371.703]
)

In [ ]:
def apply_immune_colors():
    import seaborn as sns
    shading_mode = be.SHADING_MODE_BASIC
    specular_exponent = 50
    for model in core.scene.models:
        model_id = model['id']
        model_name = model['name']

        material_ids = be.get_material_ids(model_id)['ids']
        nb_materials = len(material_ids)

        if be.NAME_GLUCOSE in model_name:
            palette = sns.color_palette('Blues', nb_materials)
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids, palette=palette)

        if be.NAME_LACTOFERRIN in model_name:
            palette = sns.color_palette('afmhot', nb_materials)
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids, palette=palette)

        if be.NAME_DEFENSIN in model_name:
            palette = sns.color_palette('plasma_r', nb_materials)
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids, palette=palette)

        if be.NAME_SURFACTANT_HEAD in model_name:
            palette = sns.color_palette('OrRd_r', nb_materials)
            emission = 0
            be.set_materials_from_palette(
                shading_mode=shading_mode, specular_exponent=specular_exponent,
                model_ids=[model_id], material_ids=material_ids,
                palette=palette, emission=emission)
        if be.NAME_COLLAGEN in model_name:
            material_ids = list(be.get_material_ids(model_id)['ids'])
            nb_materials = len(material_ids)
            palette = list()
            emissions = list()
            for i in range(nb_materials):
                palette.append([1,1,1])
                emissions.append(0.1)
            be.set_materials(
                model_ids=[model_id], material_ids=material_ids,
                diffuse_colors=palette, specular_colors=palette,
                emissions=emissions
            )

In [ ]:
set_all_materials_to_grey()
apply_immune_colors()

In [ ]:
mm.create_snapshot(
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/01600.png')

### Frame 2600 - Surfactant head

In [ ]:
status = core.set_camera(
    orientation=[0.086, 0.475, -0.055, 0.873],
    position=[-9.827, 110.720, 60.944],
    target=[-10.648, 110.923, 60.410]
)

In [ ]:
mm.create_snapshot(
    size=image_size, samples_per_pixel=image_samples_per_pixel,
    path=output_folder + '/02600.png')